# Computes SPEAR-MED Z-Scores
We re-compute z-scores for all variables of interest for SPEAR-MED ensemble members. We standardize the format so that all calculations of historical and future scenario data is formatted identically. Embarrassingly parallel is used save time as ensemble members can be treated independently.

In [1]:
from imports import *
from netCDF4 import Dataset # for saving netcdfs 
from cartopy.feature import ShapelyFeature # for plotting HUC2 regions
from cartopy.io.shapereader import Reader
from joblib import Parallel, delayed

In [3]:
# Functions for computing historical zscores
def is_winter(month):
    return (month <=4) | (month >=10)

def ZSCORES_historical(hist_ar, lat, lon):
    """Crank ZSCORES in numpy"""
    # get slices
    hist_slice = hist_ar[:, lat, lon]
    # get empirical distribution based on historical
    dist = ECDF(hist_slice, side='right')
    # rank historical snowfall
    INDEX = dist(hist_slice)
    # clean 0s and 1s - which result in infs under inverse normal transformation 
    try:
        min_value, max_value = dist.y[1], dist.y[-2]  # base thresholds on historical distribution
        # clean up infs (0s and 1s)
        INDEX = np.where(INDEX < min_value, min_value, INDEX)
        INDEX = np.where(INDEX > max_value, max_value, INDEX)
    except ValueError:
        pass
    
    # return indices as z-scores
    return norm.ppf(INDEX)

def hist_monthly_ZSCORES(hist_ar):
    """Computes Zscores over all gridcells"""
    # copy array for saving data
    future_ZSCORES = hist_ar.copy()
    # get dims
    nlats = hist_ar.shape[1]
    nlons = hist_ar.shape[2]
    
    # iterate through array
    for lat in range(nlats):
        for lon in range(nlons):
            
            # compute gridpoint level SWEI
            ZSCORES = ZSCORES_historical(hist_ar, lat, lon)
            future_ZSCORES[:, lat, lon] = ZSCORES # add to array
            
    return future_ZSCORES

def historical_zscores(hist_ens, variable, path_var):
    """Takes files and computes SWEI for historical dataset. """
    
    # constants - easier if wrapped in function 
    lat_new = np.arange(32, 52, 0.5)
    lon_new = np.arange(235, 255, 0.5)
    
    # load historical ds
    historical = xr.open_mfdataset(hist_ens)
    
    # process to correct grid-cells
    historical = historical.reindex(lat=lat_new, lon=lon_new, method="nearest")
    
    # cycle through months of year
    months = []
    for month in range(1,13):
        
        # extract snow values and save to numpy for faster computation
        hst = historical.sel(time=historical["time.month"]==month)
        
        hist_ar  = np.array(hst[variable])
        
        # process to Z-scores values
        ZSCORES  = hist_monthly_ZSCORES(hist_ar)
        
        # format array as month 
        month_xr = xr.Dataset(
                        data_vars=dict(
                            zscores=(['time', 'lat','lon'], ZSCORES)),

                        coords=dict(
                            lat=(['lat'], lat_new),
                            lon =(['lon'], lon_new),
                            time= hst.time))
        
        # append to months array
        months.append(month_xr)
        
    # concatenate arrays and return xarray dataset sorted by time
    ens_member_zscores = xr.concat(months, dim="time").sortby('time')
    
    # get ensemble number and create directory if needed to save files
    index = hist_ens.split("/")[4].split("_")[-1]
    fpath = f"/work/Julian.Schmitt/data/zscores/{path_var}/hist/hist_monthly_ens_{index}.nc"
    
    if not os.path.exists(os.path.dirname(fpath)):
        os.makedirs(os.path.dirname(fpath))
    
    # save to netcdf and return dataset 
    ens_member_zscores.to_netcdf(fpath)
    return ens_member_zscores

In [3]:
# get filenames 
snow_historical   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Hist_AllForc_IC1921_K50/"
                         "pp_ens_*/land/ts/monthly/94yr/land.192101-201412.snow.nc")
precip_historical = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Hist_AllForc_IC1921_K50/"
                         "pp_ens_*/atmos/ts/monthly/94yr/atmos.192101-201412.precip.nc")
temp_historical   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Hist_AllForc_IC1921_K50/pp_ens_*/"
                         "atmos/ts/monthly/94yr/atmos.192101-201412.t_ref.nc")
tmin_historical   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Hist_AllForc_IC1921_K50/pp_ens_*/atmos/"
                         "ts/monthly/94yr/atmos.192101-201412.t_ref_min.nc")
tmax_historical   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Hist_AllForc_IC1921_K50/pp_ens_*/atmos/"
                         "ts/monthly/94yr/atmos.192101-201412.t_ref_max.nc")

# SSP585 - max forcing 
snow_future   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP585_IC2011_K50/"
                     "pp_ens_*/land/ts/monthly/86yr/land.201501-210012.snow.nc")
precip_future = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP585_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.precip.nc")
temp_future   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP585_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.t_ref.nc")
tmax_future   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP585_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.t_ref_max.nc")
tmin_future   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP585_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.t_ref_min.nc")

# SSP245 - medium forcing
snow_future245   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP245_IC2011_K50/"
                     "pp_ens_*/land/ts/monthly/86yr/land.201501-210012.snow.nc")
precip_future245 = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP245_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.precip.nc")
temp_future245   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP245_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.t_ref.nc")
tmax_future245   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP245_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.t_ref_max.nc")
tmin_future245   = glob("/decp/SPEAR_MED/SPEAR_c192_o1_Scen_SSP245_IC2011_K50/pp_ens_*/atmos/ts"
                     "/monthly/86yr/atmos.201501-210012.t_ref_min.nc")


In [13]:
%%time
# compute zscores for historical datasets
ncores = os.cpu_count()
hist_snow_ds   = Parallel(n_jobs=ncores)(delayed(historical_zscores)(snow_historical[i], "snow", "snow") 
                                                                                    for i in range(30))
print("Computed Snow ZScores")
hist_precip_ds = Parallel(n_jobs=ncores)(delayed(historical_zscores)(precip_historical[i], "precip", "precip") 
                                                                                    for i in range(30))
print("Computed Precip ZScores")
hist_temp_ds   = Parallel(n_jobs=ncores)(delayed(historical_zscores)(temp_historical[i], "t_ref", "temp") 
                                                                                    for i in range(30))
print("Computed Temp ZScores")
hist_tmin_ds   = Parallel(n_jobs=ncores)(delayed(historical_zscores)(tmin_historical[i], "t_ref_min", "tmin") 
                                                                                    for i in range(30))
print("Computed Tmin ZScores")
hist_tmax_ds   = Parallel(n_jobs=ncores)(delayed(historical_zscores)(tmax_historical[i], "t_ref_max", "tmax") 
                                                                                    for i in range(30))
print("Computed Tmax Zscores")

Computed Snow ZScores
Computed Precip ZScores
Computed Temp ZScores
Computed Tmin ZScores
Computed Tmax Zscores
CPU times: user 2.35 s, sys: 1.08 s, total: 3.43 s
Wall time: 8min 53s


## Compute ZScores for future conditions
We base the future zscores off of the entire historical record. 

In [4]:
def ZSCORES_from_historical(hist_ar, future_ar, lat, lon):
    """Compute ZSCORES in numpy for future datasets based on historical"""
    # get slices
    hist_slice = hist_ar[:, lat, lon]
    future_slice = future_ar[:, lat, lon]
    # get empirical distribution based on historical
    dist = ECDF(hist_slice, side='right')
    # rank values for future snowfall
    INDEX = dist(future_slice)
    # clean 0s and 1s - which result in infs under inverse normal transformation 
    try:
        min_value, max_value = dist.y[1], dist.y[-2]  # base thresholds on historical distribution - eg remove infs 
        INDEX = np.where(INDEX < min_value, min_value, INDEX)
        INDEX = np.where(INDEX > max_value, max_value, INDEX)
    except ValueError:
        pass
    
    # return indices as z-scores
    return norm.ppf(INDEX)

def month_ZSCORES(hist_ar, future_ar):
    # copy array for saving data
    future_ZSCORES = future_ar.copy()
    # get dims
    nlats = hist_ar.shape[1]
    nlons = hist_ar.shape[2]
    # iterate through array
    for lat in range(nlats):
        for lon in range(nlons):
            # compute gridpoint level SWEI
            ZSCORES = ZSCORES_from_historical(hist_ar, future_ar, lat, lon)
            # add to array
            future_ZSCORES[:, lat, lon] = ZSCORES
    return future_ZSCORES

def future_zscores(hist_ens, ssp_ens, variable, path_var):
    """Takes files and computes SWEI for future dataset. """
    # constants - easier if wrapped in function 
    lat_new = np.arange(32, 52, 0.5)
    lon_new = np.arange(235, 255, 0.5)
    
    # load files
    historical = xr.open_mfdataset(hist_ens)
    future     = xr.open_mfdataset(ssp_ens)
    
    # process to correct grid-cells
    historical = historical.reindex(lat=lat_new, lon=lon_new, method="nearest")
    future     = future.reindex(lat=lat_new, lon=lon_new, method="nearest")
    
    # cycle through months of year
    months = []
    for month in range(1,13):
        
        # extract snow values and save to numpy for faster computation
        hst = historical.sel(time=historical["time.month"]==month)
        ftr = future.sel(time=future['time.month']==month)
        
        hist_ar   = np.array(hst[variable])
        future_ar = np.array(ftr[variable])
        
        # process to Z-scores values
        ZSCORES = month_ZSCORES(hist_ar, future_ar)
        
        # format array as month and transform to xarray dataset object
        month_xr = xr.Dataset(
                        data_vars=dict(
                            zscores=(['time', 'lat','lon'], ZSCORES)),

                        coords=dict(
                            lat=(['lat'], lat_new),
                            lon =(['lon'], lon_new),
                            time= ftr.time))
        
        # append to months array
        months.append(month_xr)
        
    # concatenate arrays and return xarray dataset sorted by time
    ens_member_zscores = xr.concat(months, dim="time").sortby('time')
    
    # get ensemble number and create directory if needed to save files
    index = hist_ens.split("/")[4].split("_")[-1]
    fpath = f"/work/Julian.Schmitt/data/zscores/{path_var}i/future245/future_monthly_ens_{index}.nc"
    
    if not os.path.exists(os.path.dirname(fpath)):
        os.makedirs(os.path.dirname(fpath))
    
    # save to netcdf and return dataset 
    ens_member_zscores.to_netcdf(fpath)
    return ens_member_zscores

In [18]:
# compute zscores for future values
ncores = os.cpu_count() # use all available cores
future_snow_ds   = Parallel(n_jobs=ncores)(delayed(future_zscores)(snow_historical[i], snow_future[i], "snow", "snow") 
                                                                                    for i in range(30))
print("Computed Snow ZScores")
future_precip_ds = Parallel(n_jobs=ncores)(delayed(future_zscores)(precip_historical[i], precip_future[i], "precip", "precip") 
                                                                                    for i in range(30))
print("Computed Precip ZScores")
future_temp_ds   = Parallel(n_jobs=ncores)(delayed(future_zscores)(temp_historical[i], temp_future[i], "t_ref", "temp") 
                                                                                    for i in range(30))
print("Computed Temp ZScores")
future_tmin_ds   = Parallel(n_jobs=ncores)(delayed(future_zscores)(tmin_historical[i], tmin_future[i], "t_ref_min", "tmin") 
                                                                                    for i in range(30))
print("Computed Tmin ZScores")
future_tmax_ds   = Parallel(n_jobs=ncores)(delayed(future_zscores)(tmax_historical[i], tmax_future[i], "t_ref_max", "tmax") 
                                                                                    for i in range(30))
print("Computed Tmax ZScores")

Computed Snow ZScores
Computed Precip ZScores
Computed Temp ZScores
Computed Tmin ZScores
Computed Tmax ZScores


In [5]:
# Compute zscores for medium forcing levels
ncores = os.cpu_count() # use all available cores
future_snow_ds245   = Parallel(n_jobs=ncores)(delayed(future_zscores)(snow_historical[i], snow_future245[i], "snow", "snow") 
                                                                                    for i in range(30))
print("Computed Snow ZScores")
future_precip_ds245 = Parallel(n_jobs=ncores)(delayed(future_zscores)(precip_historical[i], precip_future245[i], "precip", "precip") 
                                                                                    for i in range(30))
print("Computed Precip ZScores")
future_temp_ds245   = Parallel(n_jobs=ncores)(delayed(future_zscores)(temp_historical[i], temp_future245[i], "t_ref", "temp") 
                                                                                    for i in range(30))
print("Computed Temp ZScores")
future_tmin_ds245   = Parallel(n_jobs=ncores)(delayed(future_zscores)(tmin_historical[i], tmin_future245[i], "t_ref_min", "tmin") 
                                                                                    for i in range(30))
print("Computed Tmin ZScores")
future_tmax_ds245   = Parallel(n_jobs=ncores)(delayed(future_zscores)(tmax_historical[i], tmax_future245[i], "t_ref_max", "tmax") 
                                                                                    for i in range(30))
print("Computed Tmax ZScores")

Computed Snow ZScores
Computed Precip ZScores
Computed Temp ZScores
Computed Tmin ZScores
Computed Tmax ZScores
